In [110]:
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from nltk.stem import WordNetLemmatizer , PorterStemmer
stop_words = set(stopwords.words('english'))

In [111]:
sentence = "The system should be able to check authorization rules when users makes request to access server."
sentence = sentence.lower()
print(sentence)

the system should be able to check authorization rules when users makes request to access server.


In [112]:
wordsList = word_tokenize(sentence)   
wordsList = [w for w in wordsList if not w in stop_words]  
print(wordsList)

['system', 'able', 'check', 'authorization', 'rules', 'users', 'makes', 'request', 'access', 'server', '.']


In [113]:
lemmatizer = WordNetLemmatizer() 
lemma_word = list()
for e in wordsList:
    lemma_word.append(lemmatizer.lemmatize(e))
print(lemma_word)

['system', 'able', 'check', 'authorization', 'rule', 'user', 'make', 'request', 'access', 'server', '.']


In [114]:
#2 score common word in each grammar
_access = ['access','approach','gain','entry']
#3 score
_deny   = ['refuse','deny','reject','decline']
_permit = ['allow' ,'permit' ,'authorize','grant','approve']

#access  , permission
au1 = { 1: ['user','can'],
        2: _access, 
        5: ['permission'] } 

#deny , unauthorized
au2 = { 1: ['system'], 
        3: _deny + ['access'], 
        5: ['unauthorized']}   

#permit , authorized
au3 = { 1: ['system'], 
        3: _permit, 
        5: ['authorized','access']}


#access , role
_role = ['manager','admin']
RBAC  = { 1: ['user'],
          2: _access,
          3: ['obtain','role'],
          5: _role }

#access , level
_level = ['high','medium' ,'low']
_data_level = ['public','sensitive','restrictive']

MAC = { 1: ['user'],
        2: _access,
        3: _level,
        5: _data_level}

#access , rule ,inspect
_inspect = ['check','inspect','examine','consider']

RM = { 1:['system','user'], 
       2: _access + ['authorization','rule','request'],
       3:_inspect} 

#CUDE , define , OBJ
_cude = ['create' ,'update','delete','edit']
_define = ['assign','determine','establish','specify','charify','prescribe']
_obj = ['role','user','rule','right','authorization']

RR = { 1:['administrator'],
       2: _obj ,
       3: _cude +_define }

In [115]:
lemma_word

['system',
 'able',
 'check',
 'authorization',
 'rule',
 'user',
 'make',
 'request',
 'access',
 'server',
 '.']

In [116]:
score_each_grammar = {'au1':0 ,'au2':0 ,'au3':0,'RBAC':0,'MAC':0,'RM':0,'RR':0}
grammar = [('au1',au1),('au2',au2),('au3',au3),('RBAC',RBAC),('MAC',MAC),('RM',RM),('RR',RR)]

In [117]:
for w in lemma_word:
    for ng , dl in grammar:
        for score in dl :
            if w in dl[score]:
                score_each_grammar[ng] = score_each_grammar[ng] + score

In [118]:
display(lemma_word)
score_each_grammar

['system',
 'able',
 'check',
 'authorization',
 'rule',
 'user',
 'make',
 'request',
 'access',
 'server',
 '.']

{'au1': 3, 'au2': 4, 'au3': 6, 'RBAC': 3, 'MAC': 3, 'RM': 13, 'RR': 6}

In [119]:
def ingrammar(score_each_grammar):
    max_scre_grammar = -1
    grammar_key = ''
    for k in score_each_grammar:
        if(score_each_grammar[k] > max_scre_grammar):
            max_scre_grammar = score_each_grammar[k]
            grammar_key = k
    return grammar_key


In [144]:
def lemmatize_sentence(sentence):
    sentence = sentence.lower()
    wordsList = word_tokenize(sentence)   
    wordsList = [w for w in wordsList if not w in stop_words] 
    lemmatizer = WordNetLemmatizer() 
    lemma_word = list()
    for e in wordsList:
        lemma_word.append(lemmatizer.lemmatize(e))
    return lemma_word
    
def calculate_score(score_each_grammar,sentence,grammar):
    for w in sentence:
        for ng , dl in grammar:
            for score in dl :
                if w in dl[score]:
                    score_each_grammar[ng] = score_each_grammar[ng] + score


In [151]:
def grammar(sentence):
    score_each_grammar = {'au1':0 ,'au2':0 ,'au3':0,'RBAC':0,'MAC':0,'RM':0,'RR':0}
    grammar = [('au1',au1),('au2',au2),('au3',au3),('RBAC',RBAC),('MAC',MAC),('RM',RM),('RR',RR)]
    lemma_word = lemmatize_sentence(sentence)
    calculate_score(score_each_grammar,lemma_word,grammar)
    grammar_pattern = ingrammar(score_each_grammar)
    if(score_each_grammar[grammar_pattern] < 7): return 'NOT MATCH IN OUR PATTERN'
    return grammar_pattern 

In [133]:
# grammar("The system should be able to check authorization rules when users makes request to access server.")

In [134]:
# grammar('The administrator has the right to define authorization rules.')

In [135]:
# grammar('Users who have a high clearance level can access server which have a high classification level')

In [136]:
# grammar('Users who obtain engineer role can access server.')

In [137]:
# grammar('The system shall be able to permit authorized access.')

In [138]:
# grammar('The system shall be able to deny unauthorized access.')

In [139]:
# grammar('Users can access data for which they have permissions.')

In [158]:
sentences = "Users can access data for which they have permissions.\
                The system shall be able to deny unauthorized access.\
                The system shall be able to permit authorized access.\
                Users who obtain engineer role can access server.\
                Users who have a high clearance level can access server which have a high classification level.\
               The system should be able to check authorzation rules when users makes request to access server.\
               The system shall be able to keep logging to data log"
listsent = sent_tokenize(sentences)
print(len(listsent))
display(listsent)

7


['Users can access data for which they have permissions.',
 'The system shall be able to deny unauthorized access.',
 'The system shall be able to permit authorized access.',
 'Users who obtain engineer role can access server.',
 'Users who have a high clearance level can access server which have a high classification level.',
 'The system should be able to check authorzation rules when users makes request to access server.',
 'The system shall be able to keep logging to data log']

In [159]:
check_list = {'au1':0 ,'au2':0 ,'au3':0,'RBAC':0,'MAC':0,'RM':0,'RR':0}
for sentence in listsent:
    check_list[grammar(sentence)] = 1
    print( grammar(sentence) + " : " + sentence)

au1 : Users can access data for which they have permissions.
au2 : The system shall be able to deny unauthorized access.
au3 : The system shall be able to permit authorized access.
RBAC : Users who obtain engineer role can access server.
MAC : Users who have a high clearance level can access server which have a high classification level.
RM : The system should be able to check authorzation rules when users makes request to access server.
NOT MATCH IN OUR PATTERN : The system shall be able to keep logging to data log


In [162]:
for k , v in check_list.items() :
    if v == 0:
        print(k + ' is missing pattern')

RR is missing pattern
